In [1]:
#Programa inteligente que aprende a jugar "Yan ken po"

options = ["piedra", "tijeras", "papel"]

In [2]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
    return result

In [3]:
search_winner("papel", "tijeras")

2

In [4]:
from random import choice
def get_choice():
    return choice(options)

In [5]:
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("player1: %s player2: %s Winner: %s " % (
        player1, player2, search_winner(player1, player2)
    ))

player1: tijeras player2: tijeras Winner: 0 
player1: papel player2: piedra Winner: 1 
player1: piedra player2: piedra Winner: 0 
player1: papel player2: piedra Winner: 1 
player1: papel player2: tijeras Winner: 2 
player1: tijeras player2: tijeras Winner: 0 
player1: piedra player2: piedra Winner: 0 
player1: piedra player2: papel Winner: 2 
player1: tijeras player2: tijeras Winner: 0 
player1: tijeras player2: tijeras Winner: 0 


In [6]:
def str_to_list(option):
    if option=="piedra":
        res = [1,0,0]
    elif option=="tijeras":
        res = [0,1,0]
    else:
        res = [0,0,1]
    return res

data_X = list(map(str_to_list, ["piedra", "tijeras", "papel"]))
data_y = list(map(str_to_list, ["papel", "piedra", "tijeras"]))
print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [7]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(verbose=False, warm_start=True) #Neural network Model
model

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=True)

In [8]:
#Entreamiento con un solo juego : Papel[0, 0, 1] gana a Piedra [1, 0, 0]
model.fit([data_X[0]], [data_y[0]]) # Training the Neural network model

C:\python\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=True)

In [13]:
print(model.predict_proba([str_to_list("piedra")]) ) # [1, 0, 0]
print(model.predict_proba([str_to_list("tijeras")])) # [0, 1, 0]
print(model.predict_proba([str_to_list("papel")])  ) # [0, 0, 1]

[[0.02632174 0.02580255 0.97593528]]
[[0.13827418 0.12048239 0.8537476 ]]
[[0.12640879 0.11497652 0.85197876]]


In [15]:
#juega y aprende
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        
        predict = model.predict_proba([str_to_list(player1)])[0]
        
        if predict[0] >= 0.95:
            player2 = options[0]
        elif predict[1] >= 0.95:
            player2 = options[1]
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
            
        if debug==True:
            print("Player1: %s Player2 (modelo): %s --> %s" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)
        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
        
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            
            score["win"]+=1
        else:
            score["loose"]+=1
        
    return score, data_X, data_y

In [16]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("Score: %s %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)

Player1: tijeras Player2 (modelo): [0.13827418 0.12048239 0.8537476 ] --> piedra
Comprobamos: p1 VS p2: 2
[[0, 1, 0]]
[[1, 0, 0]]
Score: {'win': 1, 'loose': 0} 100.0 %


In [17]:
#Aprendizaje de la red neuronal mediante juega y aprende

i = 0
historic_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historic_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y) # Reentrenando
    
    if sum(historic_pct[-9:])==900:
        break

Iter: 1 - score: {'win': 545, 'loose': 455} 54.5 %
Iter: 2 - score: {'win': 560, 'loose': 440} 56.0 %
Iter: 3 - score: {'win': 561, 'loose': 439} 56.1 %
Iter: 4 - score: {'win': 547, 'loose': 453} 54.7 %
Iter: 5 - score: {'win': 554, 'loose': 446} 55.4 %
Iter: 6 - score: {'win': 557, 'loose': 443} 55.7 %
Iter: 7 - score: {'win': 569, 'loose': 431} 56.9 %
Iter: 8 - score: {'win': 520, 'loose': 480} 52.0 %
Iter: 9 - score: {'win': 582, 'loose': 418} 58.2 %
Iter: 10 - score: {'win': 561, 'loose': 439} 56.1 %
Iter: 11 - score: {'win': 577, 'loose': 423} 57.7 %
Iter: 12 - score: {'win': 337, 'loose': 663} 33.7 %
Iter: 13 - score: {'win': 323, 'loose': 677} 32.3 %
Iter: 14 - score: {'win': 344, 'loose': 656} 34.4 %
Iter: 15 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 16 - score: {'win': 317, 'loose': 683} 31.7 %
Iter: 17 - score: {'win': 339, 'loose': 661} 33.9 %
Iter: 18 - score: {'win': 332, 'loose': 668} 33.2 %
Iter: 19 - score: {'win': 351, 'loose': 649} 35.1 %
Iter: 20 - score: {'w

Iter: 158 - score: {'win': 572, 'loose': 428} 57.2 %
Iter: 159 - score: {'win': 555, 'loose': 445} 55.5 %
Iter: 160 - score: {'win': 574, 'loose': 426} 57.4 %
Iter: 161 - score: {'win': 559, 'loose': 441} 55.9 %
Iter: 162 - score: {'win': 561, 'loose': 439} 56.1 %
Iter: 163 - score: {'win': 564, 'loose': 436} 56.4 %
Iter: 164 - score: {'win': 573, 'loose': 427} 57.3 %
Iter: 165 - score: {'win': 517, 'loose': 483} 51.7 %
Iter: 166 - score: {'win': 774, 'loose': 226} 77.4 %
Iter: 167 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 168 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 169 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 170 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 171 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 172 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 173 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 174 - score: {'win': 1000, 'loose': 0} 100.0 %
Iter: 175 - score: {'win': 1000, 'loose': 0} 100.0 %


In [18]:
from bokeh.plotting import figure, show
from bokeh.io import push_notebook, show, output_notebook
output_notebook()

Loading BokehJS ...

In [20]:
x = range(len(historic_pct))
y = historic_pct

p = figure(
    title="Porcetaje de aprendizaje en cada iteración",
    x_axis_label="Iter", y_axis_label="%", width=900)

p.line(x, y,  line_width=1)
#p.line(x, y, legend=None, line_width=1)
show(p)